In [2]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.plotly as py
import plotly
import statsmodels.api as sm
from scipy.stats import skewnorm as skn
from scipy.stats import norm
import statsmodels.api as sm
import plotly.graph_objs as go
import plotly.figure_factory as ff

# Using plotly api_key credentials
# plotly.tools.set_credentials_file(username='aspiringfastlaner', api_key='')
# Pulling all historical data and collapsing into raw dataframe for use
from rawdatapull import datacollect

C:\Users\Fang\Desktop\Python Trading\SPX Option Backtester\SPX Put Selling\Risk Tools\rawdatapull.py:109: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### User-Defined functions

Below cell holds all necessary functions for VaR calculations
- latest_yahoo: Pulls latest yahoo data for SPX, VIX, VVIX, and SKEW

In [3]:
# Pulling Yahoo live data

'''
Function for pulling latest SPX, VIX, VVIX, or SKEW data. Input is a string, pulls 
the latest 2 lines of data from yahoo finance for given ticker and returns a 
dataframe of the open and close with the latest date as the first row.
'''
def latest_yahoo(ticker = 'SPX'):
    # Using requests to ping yahoo finance to retrieve 
    # historical data table
    if ticker == 'VIX':
        site = 'https://finance.yahoo.com/quote/%5EVIX/history?p=^VIX'
    elif ticker == 'VVIX':
        site = 'https://finance.yahoo.com/quote/%5EVVIX/history?p=^VVIX'
    elif ticker == 'SKEW':
        site = 'https://finance.yahoo.com/quote/%5ESKEW/history?p=^SKEW'
    else:
        site = 'https://finance.yahoo.com/quote/%5EGSPC/history?p=^GSPC'
        
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    # Initializing list to store date, open, and close values
    # for GSPC
    dates = []
    opens = []
    closes = []
    
    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    end_row = 3
    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        
        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = [item.split('>')[1] for item in [string.split('</span>')[0] for string in individual_row[0].split('<span ')[1:]]]
        
        if i == 1:
            # Skip first row because they are column headers
            i += 1
            continue
        elif i == end_row:
            break
        else:
            # Append necessary items to initialized lists for 
            # dataframe storage
            dates.append(row_items[0])
            opens.append(float(row_items[1].replace(',','')))
            closes.append(float(row_items[5].replace(',','')))
        i += 1
    
    # Return dataframe of necessary values
    return pd.DataFrame({ticker + ' Open': opens,ticker + ' Close': closes}, index = dates)

'''
Helper function to pull all relevant current data from yahoo finance using
latest_yahoo function call
'''
def yahoo_latest_data():
    spx_current = latest_yahoo()['SPX Close'][0]
    vix_current = latest_yahoo('VIX')['VIX Close'][0]
    skew_current = latest_yahoo('SKEW')['SKEW Close'][0]
    vvix_current = latest_yahoo('VVIX')['VVIX Close'][0]
    return spx_current, vix_current, skew_current, vvix_current

'''
Function for generating html table in Dash
'''
def generate_table(dataframe):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(len(dataframe))]
    )

'''
Functions for estimating put value
iv_predict(): takes in a vixlevel, days to expiry, and spx std and outputs an estimated vix change given the spx std.
the estimation is done via linear regression of the vix return against the SPX return std. This function is used 
to approximate the vega impact of a X std down move of the SPX on the put option price.
'''
def iv_predict(vixlevel, dte, spxstd):
    # Creating training set to estimate IV Change
    iv = datacollect.df['VIX Close']/np.sqrt(252/dte)/100
    iv.name = 'IV DTE'
    vix_ret = datacollect.df['VIX Close'].apply(pd.to_numeric).pct_change(dte)
    vix_ret.name = 'VIX Return'
    spx_ret = datacollect.df['SPX Close'].apply(pd.to_numeric).pct_change(dte)
    spx_ret.name = 'SPX Return'
    trainingset = pd.concat([iv,vix_ret,spx_ret],axis = 1).dropna()
    trainingset['SPX Std'] = trainingset['SPX Return']/trainingset['IV DTE']
    
    linreg = sm.OLS(trainingset['VIX Return'], trainingset['SPX Std']).fit()
    iv_change = linreg.predict(spxstd)[0]
    return ((1-iv_change)*vixlevel - vixlevel)/100

'''
put_delta_to_strike: takes in the greeks as well as the spot and strike price for the PUT option and approximates
what the change in put price if the SPX spot falls to the strike price. The approximation uses delta-gamma-vega-theta
approximation along with estimating the iv change should such a drop happen.
'''
def put_delta_to_strike(spot,strike,delta,gamma,vega,theta,dte,vixlevel):
    spxstd = (strike/spot-1)/(vixlevel/(100*np.sqrt(252/dte)))
    iv_change = iv_predict(vixlevel, dte, abs(spxstd))
    spxchange = strike - spot
    putprice = delta*spxchange + 0.5*gamma*(spxchange**2) + vega*iv_change + theta*dte
    return putprice

'''
put_delta: similar to put_delta_to_strike, however, this time it takes in the SPX std instead of the strike to approximate
what the change in put price would be if the SPX spot falls that many std. The approximation also uses delta-gamma-vega-theta
approximation.
'''
def put_delta(spot,spxstd,delta,gamma,vega,theta,dte,vixlevel):
    spxchange = -spot*(spxstd*(vixlevel/(100*np.sqrt(252/dte))))
    iv_change = iv_predict(vixlevel, dte, abs(spxstd))
    putprice = delta*spxchange + 0.5*gamma*(spxchange**2) + vega*iv_change + theta*dte
    return putprice

# Importing necessary models
import pandas as pd
import time
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.greeks.analytical import delta as bsdelta

# Main function for pulling data 
# Function is default set to ^GSPC ticker but can pull
# any stocks with options chains on yahoo. Note that
# a hack solution for unstacking the multi-index
# dataframe was created by saving down to csv
# and then reading back into function.
def get_options_data(ticker = '^GSPC'):
    tape = Options(ticker, 'yahoo')
    data = tape.get_all_data()
    data.to_csv('raw.csv')
    data = pd.read_csv('raw.csv')
    del data['JSON']
    return data

'''
Function for pulling latest monthly yahoo options data to create a dataframe for 
plotting. Inputs take the lower bound and upper bound contract delta filters.
'''
def options_chain(delta_lb, delta_ub):

    options_data = get_options_data()
    options_data = options_data[options_data['Type'] == 'put']
    options_data['DTE'] = (pd.to_datetime(options_data['Expiry']) - dt.datetime.today()).apply(lambda x: int(x.days))
    options_data['Delta'] = np.nan
    options_data['Strike Return'] = np.nan
    options_data['Yield'] = np.nan
    options_data = options_data.sort_values('DTE', ascending = True)
    options_data = options_data[options_data['DTE'] == options_data['DTE'].iloc[0]]
    options_data = options_data.sort_values('Strike', ascending = True).reset_index()
    del options_data['index']
    spx = latest_yahoo()

    for index, row in options_data.iterrows():
        S = float(spx['SPX Close'][0])
        K = float(row['Strike'])
        r = .01
        t = float(row['DTE'])/365
        q = 0
        sigma = float(row['IV'])
        flag = 'p'
        options_data.iloc[index,19] = bsdelta(flag, S, K, t, r, sigma, q)
        options_data.iloc[index,20] = round(K/S - 1,4)
        options_data.iloc[index,21] = (row['Last']/S)*(365/23)

    options_data = options_data[(options_data['Delta'] > delta_lb) & (options_data['Delta'] < delta_ub)][['Strike',
                                                                                                          'Expiry',
                                                                                                            'Last',
                                                                                                           'Bid',
                                                                                                           'Ask',
                                                                                                           'IV',
                                                                                                           'Delta',
                                                                                                           'Strike Return',
                                                                                                           'Yield']]

    return options_data

'''
Function for calculating the single day implied VaR for the SP 500 index
using VIX, SKEW, and the SPX spot.
Inputs:
 - rolling_window [int] - for the number of days to expiry of put option
 - var_pct [float] - for the VaR level
 - option [string of length 1] - for put or call
 - dayformat [string] - 'calendar' or 'trading' days
'''
def implied_spx_var(rolling_window, var_pct, dayformat = 'trading', option = 'P'):
    spx, vix, skew, vvix = yahoo_latest_data()
    
    alpha = -(skew - 100)/10
    if dayformat == 'calendar':
        period_vix = (np.sqrt(((vix*vix)/365)*1.5)/100)*np.sqrt(rolling_window)
    else:
        period_vix = vix*np.sqrt(rolling_window/252)/100
        
    if option == 'C':
        var_pct = 1 - var_pct
        pct_var = norm.ppf(var_pct, 0, period_vix)
    else:
        pct_var = skn.ppf(var_pct, alpha, 0, period_vix)
    strike_suggestion = spx*np.exp(pct_var)#(1 + pct_var)
    # print('VaR return percent for SPX is: ' + str(round(pct_var*100,2)))
    # print('Suggested SPX strike: ' + str(np.floor(spx_k_suggestion)))
    var_spx_return = str(round(pct_var*100,2))
    
    return strike_suggestion, var_spx_return

'''
Function for finding the historical worst returns for SPX given the current
VIX level.
Inputs:
 - vixlvl [float]: the level of VIX to filter data on
 - dte [int]: the number of days until option expiry
 - price ['Open', 'Close']: optional setting to either checking Open or Close SPX prices

Output:
 - Function returns a filtered and calculated dataframe with the worst returns
   for the given dte specified in ascending order.
'''
def worst_return(vixlvl, dte, price = 'Close'):
    temp_df = datacollect.df[['SPX Open','SPX Close','VIX Open','VIX Close']].apply(pd.to_numeric)
#     skew = datacollect.skew_raw
#     skew.columns = ['Date','Skew','na1','na2']
#     skew = skew[1:]
#     skew['Date'] = pd.to_datetime(skew['Date'])
#     skew = skew.set_index(pd.DatetimeIndex(skew['Date']))[['Skew']]
#     skew = skew.apply(pd.to_numeric)
#     temp_df = pd.concat([temp_df,skew],axis = 1)
    
    temp_df['spx_shift'] = temp_df['SPX Close'].shift(-dte)
    temp_df['vix_shift'] = temp_df['VIX Close'].shift(-dte)

    if price == 'Open':
        temp_df['Percent Return'] = (temp_df['spx_shift']/temp_df['SPX Open'] - 1)*100
        temp_df['RV'] = (np.log(temp_df['SPX Open'].shift(-1))-np.log(temp_df['SPX Open'])).rolling(20).std()*np.sqrt(252)*100
        temp_df['IV SPX Return'] = temp_df['Percent Return']/(temp_df['VIX Open']*np.sqrt(dte/252))
        temp_df['RV SPX Return'] = temp_df['Percent Return']/(temp_df['RV']*np.sqrt(dte/252))
        temp_df['IV RV Spread'] = temp_df['VIX Open'] - temp_df['RV']
        temp_df = temp_df[temp_df['VIX Open'] <= vixlvl].dropna()
    else:
        temp_df['Percent Return'] = (temp_df['spx_shift']/temp_df['SPX Close'] - 1)*100
        temp_df['RV'] = (np.log(temp_df['SPX Close'].shift(-1))-np.log(temp_df['SPX Close'])).rolling(20).std()*np.sqrt(252)*100
        temp_df['IV SPX Return'] = temp_df['Percent Return']/(temp_df['VIX Close']*np.sqrt(dte/252))
        temp_df['RV SPX Return'] = temp_df['Percent Return']/(temp_df['RV']*np.sqrt(dte/252))
        temp_df['IV RV Spread'] = temp_df['VIX Close'] - temp_df['RV']
        temp_df = temp_df[temp_df['VIX Close'] <= vixlvl].dropna()


    if price == 'Open':
        temp_df = temp_df[['SPX Open','VIX Open','RV','Percent Return',
                       'IV SPX Return','RV SPX Return','IV RV Spread']]
    else:
        temp_df = temp_df[['SPX Close','VIX Close','RV','Percent Return',
                       'IV SPX Return','RV SPX Return','IV RV Spread']]
        
    temp_df['Estimated SPX Level'] = latest_yahoo(ticker = 'SPX').iloc[0,0]*(1 + temp_df['Percent Return']/100)
    temp_df.columns = ['SPX','VIX','RV','Percent Return', 'IV SPX Return','RV SPX Return','IV RV Spread',
                       'Estimated SPX Level']
    
    temp_df['Date'] = temp_df.index
    temp_df = temp_df[['Date'] + temp_df.columns.tolist()[:-1]]
    return temp_df.sort_values('Percent Return', ascending = True)

### Creating Dash Application
#### First cell is setting up layout along with dash components
Layout was mirrored from: https://plot.ly/dash/gallery/volatility-surface

In [4]:
# Setup app
# server = flask.Flask(__name__)
# server.secret_key = os.environ.get('secret_key', 'secret')
# app = dash.Dash(__name__, server=server, url_base_pathname='/dash/gallery/volatility-surface', csrf_protect=False)
app = dash.Dash()

external_css = ["https://fonts.googleapis.com/css?family=Overpass:300,300i",
                "https://cdn.rawgit.com/plotly/dash-app-stylesheets/dab6f937fd5548cebf4c6dc7e93a10ac438f5efb/dash-technical-charting.css"]

for css in external_css:
    app.css.append_css({"external_url": css})

if 'DYNO' in os.environ:
    app.scripts.append_script({
        'external_url': 'https://cdn.rawgit.com/chriddyp/ca0d8f02a1659981a0ea7f013a378bbd/raw/e79f3f789517deec58f41251f7dbb6bee72c44ab/plotly_ga.js'
    })


# Plot Fields
returncolumns = ['SPX','VIX','RV','Percent Return', 'IV SPX Return','RV SPX Return','IV RV Spread']
plotfields = [dict(label=str(x), value=str(x)) for x in returncolumns]

# Current data table for graphing
curr_data = np.round(datacollect.curr_table.tail(),2)
curr_data['Date'] = datacollect.curr_table.tail().index
curr_data = curr_data.iloc[:,::-1]

# Creating suggestion strikes
rvstd = curr_data['RV'][-1]/(np.sqrt(252)*100)
ivstd = curr_data['VIX Close'][-1]/(np.sqrt(252)*100)

sugg_df = pd.DataFrame(np.round(np.vstack([np.arange(0.5,5,0.5),
                                           [(1-rvstd*std)*curr_data['SPX Close'][-1] for std in np.arange(0.5,5,0.5)],
                                           [(1-ivstd*std)*curr_data['SPX Close'][-1] for std in np.arange(0.5,5,0.5)]]),2))

sugg_df.columns = [str(x) for x in sugg_df.iloc[0,:]]
sugg_df = sugg_df.iloc[1:,:]
#sugg_df['STD Calculation'] = np.array(['RV','IV']).T

sugg_df.insert(0, 'STD Calculation', np.array(['RV','IV']).T)

# Make app layout
app.layout = html.Div(
    [# Titles
        html.Div([
            html.Img(
                src="http://fchen.info/wp-content/uploads/2016/10/fclogo2.png",
                className='five columns',
                style={
                    'height': '60',
                    'width': '60',
                    'float': 'left',
                    'text-align': 'center'
                },
            ),
            html.H1(
                'SPX Options Risk Management Tool',
                className='seven columns',
                style={'text-align': 'center'}
            ),
        ],
            className='row'
        ),
        # First Row
        html.Hr(style={'margin': '0', 'margin-bottom': '5'}),
        html.Div([
            html.Div([
                html.Label('Worst Return Table Views:'),
                html.Div([
                    html.Label('VIX Slider:'),
                    dcc.Slider(
                        marks={i: '{}'.format(i) for i in range(10,105,5)},
                        id='vix-slider',
                        min = 10,
                        max = 100,
                        value = 20,
                    ),
                ],
                className = 'row',
                style={'padding': '3%'}),
                html.Div([
                    html.Label('DTE Slider:'),
                    dcc.Slider(
                        marks={i: '{}'.format(i) for i in range(1,11)},
                        id='dte-slider',
                        min = 1,
                        max = 10,
                        value = 1,
                    )
                ],
                className = 'row',
                style={'padding': '3%'}),
                html.Div([
                    html.Label('Number of Data Points (5 to 500):'),
                    dcc.Slider(
                        id='data-slider',
                        min = 5,
                        max = 500,
                        value = 50,
                    )
                ],
                className = 'row',
                style={'padding': '3%'}),
                html.Div([
                    html.Div([
                        html.Label('X-Axis:'),
                        dcc.Dropdown(
                            id='x_dropdown',
                            options=plotfields,
                            value='VIX',
                        ),
                        html.Label('Y-Axis:'),
                        dcc.Dropdown(
                            id='y_dropdown',
                            options=plotfields,
                            value='RV',
                        ),
                        html.Label('Z-Axis:'),
                        dcc.Dropdown(
                            id='z_dropdown',
                            options=plotfields,
                            value='Percent Return',
                        )
                    ],
                        className = 'six columns',
                        style = {'padding': '1%'}),
                    html.Div([
                        html.Label('Time of Day Settings:'),
                        dcc.RadioItems(
                            id='open_close_selector',
                            options=[
                                {'label': 'Open', 'value': 'Open'},
                                {'label': 'Close', 'value': 'Close'},
                            ],
                            value='Close',
                            labelStyle={'display': 'inline-block'},
                        )
                    ],
                        className='six columns',
                    )],
                className = 'row',
                style={'padding': '1%'})
            ],
                className='four columns',
            ),
            html.Div([
                dcc.Graph(id='worst-return-scatter', style={'max-height': '450', 'height': '60vh'})
            ],
                className='eight columns'
            )],
            className='row',
            style={'margin-bottom': '1%'}
        ),
        
        ## Second Row
        html.Div([
            html.Div(children=[
                html.H4(children='Latest Market Data'),
                generate_table(curr_data)],
                     className = 'six columns',
                     style = {'text-align': 'center',
                             'margin-left': 'auto',
                             'margin-right': 'auto',
                             'align': 'center'}),
            html.Div([
                dcc.Graph(id = 'daily-return-distribution', style={'max-height': '450', 'height': '60vh'})
            ],
                className='six columns'
            )
        ],
            className='row',
            style = {'margin-bottom': '3%','text-align': 'center',
                     'margin-left': 'auto',
                     'margin-right': 'auto',
                     'align': 'center'}
        ),
        
        ## Second Row
        html.Div([
            html.Div(children=[
                html.H4(children='SPX Suggested Strikes Given IV/RV'),
                generate_table(sugg_df)],
                     style = {'text-align': 'center',
                             'margin-left': 'auto',
                             'margin-right': 'auto',
                             'align': 'center'})
        ],
            className='row',
            style = {'margin-bottom': '3%','text-align': 'center',
                     'margin-left': 'auto',
                     'margin-right': 'auto',
                     'align': 'center'}
        ),        
        
        ## Third Row
        html.Div([
            html.Div([
                html.Div([
                    html.Label('Days to Expiry:'),
                    dcc.Input(id='dte-input', type='text', value='1')
                ]),
                html.Div([
                    html.Label('VaR Threshold:'),
                    dcc.Input(id='var-input', type='text', value='0.0005')
                ])
            ],
                className='four columns'
            ),
            html.Div([
                html.Label('Trading or Calendar Days:'),
                dcc.RadioItems(
                    id='day-format-select',
                    options=[
                        {'label': 'Trading Days', 'value': 'trading'},
                        {'label': 'Calendar Days', 'value': 'calendar'},
                    ],
                    value='calendar',
                    labelStyle={'display': 'inline-block'},
                )
            ],
                className='four columns',
            ),
            html.Div([
                html.Label('Implied VaR Calculation'),
                html.P(id='var-calc-output')
            ],
                className='four columns'
            )
        ],
            className='row',
            style = {'margin-bottom': '3%'}
        ),
        
        ## Fourth Row
        html.Div([
            html.Div([
                html.Label('Option Simulated Price Change Settings:'),
                html.Div([
                    html.Div([
                        html.Label('Delta'),
                        dcc.Input(id='delta-input', type='text', value='0.10')
                    ],
                        style={'display': 'inline-block',
                               'padding': '3%'},
                        className = 'two columns'
                    ),
                    html.Div([
                        html.Label('Gamma'),
                        dcc.Input(id='gamma-input', type='text', value='0.005')
                    ],
                        style={'display': 'inline-block',
                               'padding': '3%'},
                        className = 'two columns'
                    ),                    
                    html.Div([
                        html.Label('Vega'),
                        dcc.Input(id='vega-input', type='text', value='0.05')
                    ],
                        style={'display': 'inline-block',
                               'padding': '3%'},
                        className = 'two columns'
                    ),
                    html.Div([
                        html.Label('Theta'),
                        dcc.Input(id='theta-input', type='text', value='0.5')
                    ],
                        style={'display': 'inline-block',
                               'padding': '3%'},
                        className = 'two columns'
                    ),
                    
                     html.Div([
                        html.Label('PUT Premium'),
                        dcc.Input(id='premo-input', type='text', value='1.00')
                    ],
                        style={'display': 'inline-block',
                               'padding': '3%'},
                        className = 'four columns'
                    )
                    
                ]
                ),
            ],
                className='row',
                style={'display': 'inline-block'}
            ),
            
        ],
            className='row',
            style={'margin-bottom': '20'}
        ),
        
        
        ## Fifth Row
        html.Div([
            html.Div(id = 'worst-table', className = 'four columns'),
            html.Div([
                dcc.Graph(id = 'put-delta', style={'max-height': '450', 'height': '60vh'})
            ],
                className='eight columns'
            )
        ],
            className='row',
            style={'margin-bottom': '2%'}
        ),
        
        ## Sixth Row
        html.Div([
            html.Div([
                html.Label('Lower and Upper Delta Bounds for Options Filter'),
                dcc.RangeSlider(
                        id='delta-slider',
                        marks={i: '{}'.format(round(i,1)) for i in np.arange(-1, 0, 0.1)},
                        min=-1,
                        max=-0.01,
                        step = 0.1,
                        value=[-0.2, -0.05]
                    )],
                     className = 'four columns'
                    ),
            html.Div([
                dcc.Graph(id = 'options-yield', style={'max-height': '450', 'height': '60vh'})
            ],
                className='eight columns'
            )
        ],
            className='row',
            style={'margin-bottom': '2%'}
        ),
        
        # Temporary hack for live dataframe caching
        # 'hidden' set to 'loaded' triggers next callback
        html.P(
            hidden='',
            id='worst-return-data',
            style={'display': 'none'}
        )
    ],
    style={
        'width': '85%',
        'max-width': '1200',
        'margin-left': 'auto',
        'margin-right': 'auto',
        'font-family': 'overpass',
        'background-color': '#FFFFFF',
        'padding': '40',
        'padding-top': '20',
        'padding-bottom': '20',
    },
)

# Cache raw data
# Callback function to load worst return recalculated data
# into global worst_return_data for useage
@app.callback(Output('worst-return-data', 'hidden'),
              [Input('vix-slider', 'value'),
               Input('dte-slider', 'value'),
               Input('open_close_selector', 'value'),
               Input('data-slider', 'value')])
def cache_raw_data(vix_input, dte_input, open_close_input, data_count):

    global worst_return_data
    worst_return_data = worst_return(vix_input, dte_input, open_close_input).head(data_count)
    print('Loaded raw data')

    return 'worst-return-loaded'

# Creating quick table reference for the top 5 worst returns given specified
# information 
@app.callback(Output('worst-table', 'children'), 
              [Input('x_dropdown', 'value'),
               Input('y_dropdown', 'value'),
               Input('z_dropdown', 'value')])
def worst_return_table(xval, yval, zval):
    return generate_table(pd.concat([worst_return_data['Date'],
                                     np.round(worst_return_data[[str(xval),
                                                                 str(yval),
                                                                 str(zval),
                                                                 'Estimated SPX Level']], 2)], 
                                    axis = 1).head(5))

# Main plot for dynamic scatter plot with changeable values
# All inputs are from drop down menues test
@app.callback(Output('worst-return-scatter', 'figure'), 
              [Input('worst-return-data','hidden'),
               Input('x_dropdown', 'value'),
               Input('y_dropdown', 'value'),
               Input('z_dropdown', 'value'),
               Input('vix-slider', 'value'),
               Input('dte-slider', 'value'),
               Input('open_close_selector', 'value'),
               Input('data-slider', 'value')])
def update_table(hidden, xval, yval, zval, vix_input, dte_input, open_close_input, data_count):
    temp = worst_return(vix_input, dte_input, open_close_input).head(data_count)
    trace1 = {
        "type": "scatter3d",
        'x': np.round(temp[str(xval)],2),
        'y': np.round(temp[str(yval)],2),
        'z': np.round(temp[str(zval)],2),
        'mode': 'markers',
        'marker': {'size': 5},
        'opacity': 0.7,
        'text': temp.index
    }

    layout = {
        "title": "SPX {} Scatter Plot".format(zval),
        'margin': {
            'l': 10,
            'r': 10,
            'b': 10,
            't': 60,
        },
        'paper_bgcolor': '#FFFFFF',
        "hovermode": "closest",
        "scene": {
            "aspectmode": "manual",
            "aspectratio": {
                "x": 2,
                "y": 2,
                "z": 1
            },
            'camera': {
                'up': {'x': 0, 'y': 0, 'z': 1},
                'center': {'x': 0, 'y': 0, 'z': 0},
                'eye': {'x': 1, 'y': 1, 'z': 0.5},
            },
            "xaxis": {
                "title": xval,
                "showbackground": True,
                "backgroundcolor": "rgb(230, 230,230)",
                "gridcolor": "rgb(255, 255, 255)",
                "zerolinecolor": "rgb(255, 255, 255)"
            },
            "yaxis": {
                "title": yval,
                "showbackground": True,
                "backgroundcolor": "rgb(230, 230,230)",
                "gridcolor": "rgb(255, 255, 255)",
                "zerolinecolor": "rgb(255, 255, 255)"
            },
            "zaxis": {
                "title": zval,
                "showbackground": True,
                "backgroundcolor": "rgb(230, 230,230)",
                "gridcolor": "rgb(255, 255, 255)",
                "zerolinecolor": "rgb(255, 255, 255)"
            }
        },
    }

    data = [trace1]
    figure = dict(data=data, layout=layout)
    
    print('Scatter Loaded')

    return figure


# Callback function for seeing distribution of daily return percentages given vix level
@app.callback(Output('daily-return-distribution', 'figure'), 
              [Input('vix-slider', 'value')])
def daily_return_distribution(vixlevel):
    test = datacollect.df[['SPX Close', 'VIX Close', 'Daily VIX Close']].apply(pd.to_numeric)
    test['Return'] = np.log(test['SPX Close'].shift(-1))-np.log(test['SPX Close'])
    test['RV'] = test['Return'].rolling(20).std()
    test['Daily Trading IV'] = (test['VIX Close']/(100*np.sqrt(252)))
    test = test[test['VIX Close'] <= float(vixlevel)]
    test = test.dropna()
    
    rv_breaches = []
    iv_breaches = []
    
    for std in np.arange(0.5,5,0.5):
        rv_breaches.append(sum(1*(test['Return'] <= -std*test['RV']))/len(test))
        iv_breaches.append(sum(1*(test['Return'] <= -std*test['Daily Trading IV']))/len(test))
    
    breaches = pd.DataFrame({'RV Breach': rv_breaches, 'IV Breach': iv_breaches},
                            index = np.arange(0.5,5,0.5))
    
    trace1 = go.Bar(
        x=breaches.index,
        y=breaches['IV Breach'],
        name='IV Breaches',
        marker=dict(
            color='rgb(55, 83, 109)'
        )
    )
    trace2 = go.Bar(
        x=breaches.index,
        y=breaches['RV Breach'],
        name='RV Breaches',
        marker=dict(
            color='rgb(26, 118, 255)'
        )
    )
    data = [trace1, trace2]
    layout = go.Layout(
        title='Breaches Conditioned on VIX of {}'.format(vixlevel),
        xaxis=dict(
            title = 'STD Moves',
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
        yaxis=dict(
            title='Percentage of Daily Returns',
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1
    )

    fig = go.Figure(data=data, layout=layout)
    
    return fig

# Callback function for plotting the put delta
@app.callback(Output('put-delta', 'figure'), 
              [Input('delta-input', 'value'),
               Input('gamma-input', 'value'),
               Input('vega-input', 'value'),
               Input('theta-input', 'value'),
               Input('dte-slider', 'value'),
               Input('premo-input', 'value')])
def put_delta_graph(delta, gamma, vega, theta, dte, premo):
    spot, vixlevel, skew, vvix = yahoo_latest_data()
    spxstds = np.arange(0,5,0.1)
    put_deltas = [put_delta(float(spot),
                            spxstd,
                            float(delta),
                            float(gamma),
                            float(vega),
                            float(theta),
                            int(dte), vixlevel) for spxstd in spxstds]
    put_deltas_percent = np.round(np.array(put_deltas)/float(premo),2)
    trace1 = go.Scatter(
        x = spxstds,
        y = np.round(put_deltas,2),
        mode='lines',
        name='PUT Delta Dollar'
    )
    
    trace2 = go.Scatter(
        x = spxstds,
        y = put_deltas_percent,
        mode='lines',
        name='PUT Delta Percent',
        yaxis='y2'
    )

    data = [trace1, trace2]
    layout = go.Layout(
        title = 'PUT Price Delta',
        xaxis = dict(title = 'SPX std.'),
        yaxis = dict(title = 'Put Dollar Price Change'),
        yaxis2=dict(title='Put Percent Price Change',
                    titlefont=dict(color='rgb(148, 103, 189)'),
                    tickfont=dict(color='rgb(148, 103, 189)'),
                    overlaying='y',
                    side='right'),
        hovermode='closest'
    )

    fig = go.Figure(data=data, layout=layout)
    return fig


# Callback function for yahoo latest options data
@app.callback(Output('options-yield', 'figure'),
              [Input('delta-slider', 'value')])
def update_yields(delta_slider):
    odata = options_chain(float(delta_slider[0]),
                          float(delta_slider[1]))

    trace1 = go.Scatter(
        x = odata['Strike'],
        y = odata['Yield'],
        mode='lines',
        name='PUT Delta Dollar'
    )


    data = [trace1]
    layout = go.Layout(
        title = '{0} SPX Expiry PUT Yield'.format(str(odata['Expiry'].iloc[0])),
        xaxis = dict(title = 'Strike'),
        yaxis = dict(title = 'Yield'),
        hovermode='closest'
    )

    fig = go.Figure(data=data, layout=layout)
    return fig
    
    
# Callback function for determining the VaR return of SPX given the current
# level of VIX and SKEW
@app.callback(Output('var-calc-output', 'children'),
              [Input('dte-input', 'value'),
               Input('var-input', 'value'),
               Input('day-format-select', 'value')])
def update_output(input1, input2, input3):
    strike_suggestion, var_spx_return = implied_spx_var(float(input1), float(input2), input3, option = 'P')
    
    return '''Suggested SPX Strike: {} with a VaR Return for SPX of: {}%
    '''.format(int(strike_suggestion), var_spx_return)

if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Mar/2018 13:05:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:05:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:05:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:05:05] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:05:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:05:06] "POST /_dash-update-component HTTP/1.1" 200 -


Loaded raw data


127.0.0.1 - - [23/Mar/2018 13:07:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:07:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:08:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:09:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:09:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:09:39] "POST /_dash-update-component HTTP/1.1" 200 -


Scatter Loaded


127.0.0.1 - - [23/Mar/2018 13:10:22] "POST /_dash-update-component HTTP/1.1" 200 -


Loaded raw data


127.0.0.1 - - [23/Mar/2018 13:11:05] "POST /_dash-update-component HTTP/1.1" 200 -


Scatter Loaded


127.0.0.1 - - [23/Mar/2018 13:12:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:15:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:16:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:16:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 13:17:47] "POST /_dash-update-component HTTP/1.1" 200 -


Additions:
- Include plot of SPX returns and Contango ratio against time